This notebook converts `samples.npy` to `fingerprints.npy`. After processing, check the max and mean images to make sure that you are not over- or under-cropping the data. Set `crop_rows` or `crop_cols` to `None` to see all the data.

In [8]:
data_root = 'data/drums/'
n_fft = 1024
hop_length = n_fft//4
use_logamp = False # boost the brightness of quiet sounds
reduce_rows = 10 # how many frequency bands to average into one
reduce_cols = 1 # how many time steps to average into one
crop_rows = 32 # limit how many frequency bands to use
crop_cols = 32 # limit how many time steps to use
limit = None # set this to 100 to only process 100 samples

In [9]:
%matplotlib inline
from utils import *
from tqdm import tqdm
from os.path import join
from matplotlib import pyplot as plt
from skimage.measure import block_reduce
from multiprocessing import Pool
import numpy as np
import librosa

In [10]:
%time samples = np.load(join(data_root, 'samples.npy'))

CPU times: user 1.74 ms, sys: 19.5 ms, total: 21.2 ms
Wall time: 24.5 ms


In [23]:
window = np.hanning(n_fft)
def job(y):
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    amp = np.abs(S)
    if reduce_rows > 1 or reduce_cols > 1:
        amp = block_reduce(amp, (reduce_rows, reduce_cols), func=np.mean)
    if amp.shape[1] < crop_cols:
        amp = np.pad(amp, ((0, 0), (0, crop_cols-amp.shape[1])), 'constant')
    amp = amp[:crop_rows, :crop_cols]
    if use_logamp:
        amp = librosa.logamplitude(amp**2)
    amp -= amp.min()
    if amp.max() > 0:
        amp /= amp.max()
    amp = np.flipud(amp) # for visualization, put low frequencies on bottom
    return amp
pool = Pool()
%time fingerprints = pool.map(job, samples[:limit])
fingerprints = np.asarray(fingerprints).astype(np.float32)

CPU times: user 36.4 ms, sys: 45.5 ms, total: 81.8 ms
Wall time: 798 ms


In [24]:
np.save(join(data_root, 'fingerprints.npy'), fingerprints)

In [27]:
print ('data shape:', np.shape(fingerprints))

mean = np.mean(fingerprints, axis=0)
mean -= mean.min()
mean /= mean.max()

print ('mean:')
show_array(255 * mean)
print ('max:')
show_array(255 * np.max(fingerprints, axis=0))

print ('random selection:')
indices = range(len(fingerprints))
np.random.shuffle(indices)
show_array(255 * make_mosaic(np.array(fingerprints)[indices], n=16))

data shape: (418, 32, 32)
mean:


TypeError: string argument expected, got 'bytes'

In [32]:
from io import BytesIO
import numpy as np
from PIL import Image
import IPython.display
import shutil

def show_array(a, fmt='png', filename=None):
    a = np.uint8(np.clip(a, 0, 255))
    image_data = BytesIO()
    image.fromarray(a).save(image_data, fmt)
    Image.fromarray(a).
    if filename is None:
        IPython.display.display(IPython.display.Image(data=image_data.getvalue()))
    else:
        with open(filename, 'w') as f:
            image_data.seek(0)
            shutil.copyfileobj(image_data, f)
            
show_array(255 * mean)

NameError: name 'image' is not defined